<a href="https://colab.research.google.com/github/HYUNMIN-HWANG/Colab/blob/main/DACON_vision2/vision2_0226_1_vision1train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('./MyDrive')

Drive already mounted at ./MyDrive; to attempt to forcibly remount, call drive.mount("./MyDrive", force_remount=True).


# vision1에 나온 알파벳들을 훈련시켜보자

In [4]:
# warnings.filterwarnings("ignore")
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import cv2
import gc
from keras import backend as bek
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV



In [5]:
train = pd.read_csv('/content/MyDrive/MyDrive/AIA/DACON_vision1/train.csv')
print(train.shape)  # (2048, 787)

submission = pd.read_csv('/content/MyDrive/MyDrive/AIA/DACON_vision1/submission.csv')
print(submission.shape) # (20480, 2)

test = pd.read_csv('/content/MyDrive/MyDrive/AIA/DACON_vision1/test.csv')
print(test.shape)   # (20480, 786)

(2048, 787)
(20480, 2)
(20480, 786)


In [6]:
# x
def x_preprocess (x) :
    xx = np.where((x<=20)&(x!=0) ,0.,x)  
    xx = np.where((x>=80) ,252.,x)  

    xx = xx.astype('float32')

    xx224=np.zeros([x.shape[0],56,56,3],dtype=np.float32) 

    for i, s in enumerate(xx):
        print(i)
        converted = cv2.cvtColor(s, cv2.COLOR_GRAY2RGB) 
        resized = cv2.resize(converted,(56,56),interpolation = cv2.INTER_CUBIC) 
        blur =  cv2.GaussianBlur(resized, (5,5), cv2.BORDER_DEFAULT)
        threshold, thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY) 
        # cv2.imshow("thresh", thresh)
        # cv2.waitKey(0)
        del converted
        xx224[i] = thresh
        # cv2.imshow("resized_224", x_train224[i])
        # cv2.waitKey(0)
        del thresh
        bek.clear_session()
        gc.collect()

    xx224 = xx224/255.
    print(xx224.shape)    # (2048, 56, 56, 3)
    return xx224

In [7]:
x_train = train.drop(['id', 'digit', 'letter'], axis=1).values
x_train = x_train.reshape(-1,28,28, 1)
x_train224 = x_preprocess (x_train)
print(x_train224.shape)    # (2048, 56, 56, 3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
x_test = test.drop(['id','letter'], axis=1).values
x_test = x_test.reshape(-1,28,28, 1)
x_test224 = x_preprocess (x_test)
print(x_test224.shape)   # (20480, 56, 56, 3) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
15482
15483
15484
15485
15486
15487
15488
15489
15490
15491
15492
15493
15494
15495
15496
15497
15498
15499
15500
15501
15502
15503
15504
15505
15506
15507
15508
15509
15510
15511
15512
15513
15514
15515
15516
15517
15518
15519
15520
15521
15522
15523
15524
15525
15526
15527
15528
15529
15530
15531
15532
15533
15534
15535
15536
15537
15538
15539
15540
15541
15542
15543
15544
15545
15546
15547
15548
15549
15550
15551
15552
15553
15554
15555
15556
15557
15558
15559
15560
15561
15562
15563
15564
15565
15566
15567
15568
15569
15570
15571
15572
15573
15574
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
1564

In [12]:
y1 = train['letter']
y_letter = np.zeros((len(y1),), dtype=int)  # 총 행의수 26
i = 0
for letter in y1 :
    # print(i, letter)
    # 문자열을 아스키 코드로 변환, A = 0 부터 시작하도록 지정
    asc = int( ord(letter) - 65 )
    y_letter[i] = asc
    # print(y_letter[i])
    i += 1
print(y_letter.shape)    # (2048, )
print(y_letter[:6])      # [11  1 11  3  0  2]


y2 = test['letter']
y_letter2 = np.zeros((len(y2),), dtype=int)  # 총 행의수 26
i = 0
for letter in y2 :
    # print(i, letter)
    # 문자열을 아스키 코드로 변환, A = 0 부터 시작하도록 지정
    asc = int( ord(letter) - 65 )
    y_letter2[i] = asc
    # print(y_letter[i])
    i += 1
print(y_letter2.shape)    # (20480,)
print(y_letter2[:6])      # [11  2 18 10 22  3]

(2048,)
[11  1 11  3  0  2]
(20480,)
[11  2 18 10 22  3]


In [13]:
x = np.concatenate((x_train224, x_test224), axis=0)
print(x.shape)  # (22528, 56, 56, 3)
np.save('/content/MyDrive/MyDrive/Colab/DACON_vision2/npy/vision1_x28.npy', arr=x)

y = np.concatenate((y_letter, y_letter2), axis=0)
print(y.shape)  # (22528,)
np.save('/content/MyDrive/MyDrive/Colab/DACON_vision2/npy/vision1_y28.npy', arr=y)


(22528, 56, 56, 3)
(22528,)


In [16]:
# modeling
def modeling() :
    model = Sequential()
    model.add(Conv2D(128, 3, activation='selu', padding='same', input_shape=(xx_train.shape[1], xx_train.shape[2], xx_train.shape[3])))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(32, 3, activation='selu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, 2, activation='elu', padding='same'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())

    model.add(Dense(256, activation='selu'))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='selu'))
    model.add(BatchNormalization())
    model.add(Dense(26, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.003), metrics=['acc'])
    return model

In [17]:
kf_times = 5
kf = KFold(n_splits=kf_times, shuffle=True, random_state=42)

idg = ImageDataGenerator(height_shift_range=(-1,1), 
                        width_shift_range=(-1,1),
                        rotation_range=120,
                        fill_mode='nearest'
                        )
idg2 = ImageDataGenerator()

kf_n = 1
loss_hist = []
acc_hist = []
for train_index, test_index in kf.split(x, y) :
    print("\n** Kfold %d 번째 실행 중 **" % kf_n)
    xx_train, xx_test = x[train_index], x[test_index]
    yy_train, yy_test = y[train_index], y[test_index]

    xx_train, xx_valid, yy_train, yy_valid = train_test_split(xx_train, yy_train, train_size=0.9, shuffle=True, random_state=42)
    print(xx_train.shape, xx_test.shape, xx_valid.shape)    # (16219, 56, 56, 3) (4506, 56, 56, 3) (1803, 56, 56, 3)
    print(yy_train.shape, yy_test.shape, yy_valid.shape)    # (16219,) (4506,) (1803,)

    train_generator = idg.flow(xx_train, yy_train, batch_size=16)
    test_generator = idg2.flow(xx_test, yy_test, batch_size=16)
    valid_generator = idg2.flow(xx_valid, yy_valid)

    model = modeling()

    path = '/content/MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0226_1_vison1_{val_loss:.4f}.hdf5'
    cp = ModelCheckpoint(path, monitor='val_loss', save_best_only=True, mode='min')
    rl = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, mode='min')
    es = EarlyStopping(monitor='val_loss', patience=20, mode='min')

    model.fit_generator(train_generator, epochs=1000, validation_data=valid_generator, callbacks=[cp, rl, es])

    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)

    loss_hist.append(loss)
    acc_hist.append(acc)

    kf_n +=1

print("mean loss : ", sum(loss_hist)/kf_times)
print("mean acc : ", sum(acc_hist)/kf_times)


** Kfold 1 번째 실행 중 **
(16219, 56, 56, 3) (4506, 56, 56, 3) (1803, 56, 56, 3)
(16219,) (4506,) (1803,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
1014/1014 [==============================] - 24s 20ms/step - loss: 2.6332 - acc: 0.2590 - val_loss: 1.9597 - val_acc: 0.4293
Epoch 2/1000
1014/1014 [==============================] - 19s 19ms/step - loss: 1.6060 - acc: 0.5034 - val_loss: 1.3339 - val_acc: 0.5879
Epoch 3/1000
1014/1014 [==============================] - 19s 19ms/step - loss: 1.3713 - acc: 0.5740 - val_loss: 1.1905 - val_acc: 0.6406
Epoch 4/1000
1014/1014 [==============================] - 19s 19ms/step - loss: 1.2516 - acc: 0.6061 - val_loss: 1.7347 - val_acc: 0.5352
Epoch 5/1000
1014/1014 [==============================] - 20s 19ms/step - loss: 1.1650 - acc: 0.6381 - val_loss: 2.1120 - val_acc: 0.4753
Epoch 6/1000
1014/1014 [==============================] - 19s 19ms/step - loss: 1.1277 - acc: 0.6467 - val_loss: 1.0563 - val_acc: 0.6844
Epoch 7/1000
1014/1014 [==============================] - 19s 19ms/step - loss: 1.1144 - acc: 0.6497 - val_loss: 0.9700 - val_acc: 0.7055
Epoch 8/1000
1014/1014 [==========

In [19]:
# model = load_model('../data/DACON_vision2/cp/vision_0225_1_vison1_0.3318.hdf5') # <<-- 여기에 데이콘2 테스트 데이터를 넣어서 숫자를 예측한다.

# 테스트 : 40번째 있는 문자를 예측해보자
pred_img = x_train224[49]
# pred_img = cv2.imread('../data/DACON_vision2/contour/1_1.png')
# cv2.imshow("pred_img1", pred_img)
# cv2.waitKey(0)

pred_img = pred_img.reshape(1, pred_img.shape[0], pred_img.shape[1], pred_img.shape[2])
pred_generator = idg2.flow(pred_img, shuffle=False)

result = model.predict_generator(pred_generator, verbose=True)
print(result.argmax(1))


1/1 [==============================] - 0s 157ms/step
[20]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
